In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import scipy

import matplotlib.pyplot as plt

from sklearn.base import BaseEstimator, TransformerMixin

# Helper functions

In [3]:
def change_dtypes(df):
    """
    change types of columns to reduce memory size
    :param df: dataframe
    :return df: dataframe
    """
    memory = df.memory_usage().sum() / 10**6
    print("Memory usage before changing types %0.2f MB" % memory)

    for col in df.columns:
        if (df[col].dtype == "object") and (df[col].nunique() < df.shape[0]):
            df[col] = df[col].astype("category")

        elif set(df[col].unique()) == set([0, 1]):
            df[col] = df[col].astype(bool)

        elif df[col].dtype == float:
            df[col] = df[col].astype(np.float32)

        elif df[col].dtype == int:
            df[col] = df[col].astype(np.int32)

    memory = df.memory_usage().sum() / 10 ** 6
    print("Memory usage after changing types %0.2f MB" % memory)
    return df


def load_csv(filename):
    df = pd.read_csv(filename)
    df = change_dtypes(df)
    return df

In [161]:
def check_type(val):
    if type(val) == str:
        return "string"
    
    if np.issubsctype(type(val), np.number):
        return "number"
    
    if callable(val):
        return "function"
    
    return str(type(val))


class NumColsImputer(BaseEstimator, TransformerMixin):
    def __init__(self, specified_values=None, default="median"):
        """
        :param specified_values: dict {colname (str): val (float)}, impute values for some specific columns
        :param default: str, function or float, value or function used for the remaining columns
        """
        assert (specified_values is None) or isinstance(specified_values, 
                                                        dict), "specified_values must be None or dict"
        
        self._specified_values = specified_values
        if self._specified_values is not None:
            for col, val in self._specified_values.items():
                assert check_type(val) == "number", "Impute value for " + col + " is not number."
        
        self._default = default
        self._default_type = check_type(self._default)
        if self._default_type not in ["number", "string", "function"]:
            raise ValueError("Unsupported stat type " + self._default_type)
    
    def _cal_imput_vals(self, df):
        cat_cols = df.select_dtypes(["object", "category", "bool"]).columns.to_list()
        if len(cat_cols) > 0:
            raise ValueError("There are non-number columns: " + ", ".join(cat_cols))
        
        all_cols = df.columns.to_list()
        if self._default_type == "number":
            impute_values = {col: self._default for col in all_cols}
            
        elif self._default_type == "string":
            impute_values = getattr(df, self._default)()
        
        elif self._default_type == "function":
            impute_values = df.apply(self._default)
        
        else:
            return None
            
        impute_values = dict(impute_values)
        if self._specified_values is None:
            return impute_values
        
        for col in self._specified_values:
            impute_values[col] = self._specified_values[col]
            
        return impute_values
    
    def fit(self, df):
        impute_values = self._cal_imput_vals(df)
        
        cols_with_na = [col for col in df.columns if df[col].isnull().any()]
        self._impute_values = {col: impute_values[col] for col in cols_with_na}
        
        for k, v in self._impute_values.items():
            if np.isnan(v):
                raise ValueError("One of the impute_values is NaN: " + k)
        
        return self
    
    def transform(self, df):
        return df.fillna(self._impute_values)


class CatColsImputer(BaseEstimator, TransformerMixin):
    def __init__(self, specified_values=None, default="missing_value"):
        """
        :param specified_values: dict {colname (str): val (str, float, function)}, 
                                 impute values for some specific columns
        :param default: str, used for the remaining columns
        """
        assert (specified_values is None) or isinstance(specified_values, 
                                                        dict), "specified_values must be None or dict"
        
        self._specified_values = specified_values
        if self._specified_values is not None:
            for col, val in self._specified_values.items():
                assert check_type(val) in ["string", 
                                           "function"], "Impute value for " + col + " is " + check_type(val)
        
        self._default = default
        assert check_type(self._default) == "string", "default must be string"
        
        
    def _cal_imput_vals(self, df):
        num_cols = df.select_dtypes(["number"]).columns.to_list()
        if len(num_cols) > 0:
            raise ValueError("There are number columns: " + ", ".join(num_cols))
        
        all_cols = df.columns.to_list()
        impute_values = {col: self._default for col in all_cols}
        if self._specified_values is None:
            return impute_values
        
        for col, val in self._specified_values.items():
            dtype = check_type(val)
            if dtype == "string":
                impute_values[col] = val
            
            elif dtype == "function":
                impute_values[col] = val(df[col])
            
            else:
                return None
        return impute_values
    
    def fit(self, df):
        impute_values = self._cal_imput_vals(df)
        
        cols_with_na = [col for col in df.columns if df[col].isnull().any()]
        self._impute_values = {col: impute_values[col] for col in cols_with_na}
        
        return self
    
    def transform(self, df):
        df_new = df.copy()
        for col, val in self._impute_values.items():
            df_new[col] = df_new[col].astype("object").fillna(val).astype("category")
            
        return df_new

    
class CollinearColumnRemover(BaseEstimator, TransformerMixin):
    def __init__(self, threshold, col_regex=None):
        """
        :param threshold: float in [0, 1], if two columns have correlation greater than threshold
                          one of them will be removed
        :param col_regex: str, regular expression to select columns
        """
        self._threshold = threshold
        self._col_regex = col_regex
    
    def _collinear_columns(self, df, threshold):
        if self._col_regex is None:
            df_sel = df.select_dtypes(["number", "bool"])
        else:
            df_sel = df.filter(regex=self._col_regex)
        
        all_cols = df_sel.columns.to_list()
        ncols = len(all_cols)
        
        collin_cols = []
        for i in range(ncols-1):
            col_i = all_cols[i]
            if col_i in collin_cols:
                continue
            
            for j in range(i + 1, ncols):
                col_j = all_cols[j]
                if col_j in collin_cols:
                    continue
                
                corr = df_sel[[col_i]].corrwith(df_sel[col_j]).values[0]
                if corr > threshold:
                    collin_cols.append(col_j)
        
        collin_cols = list(set(collin_cols))
        return collin_cols
    
    def _collinear_columns_NOTUSED(self, df, threshold):
        if self._col_regex is None:
            df_sel = df.select_dtypes(["number", "bool"])
        else:
            df_sel = df.filter(regex=self._col_regex)
        
        corr_matr = df_sel.corr().abs()
        upper_matr = corr_matr.where(np.triu(np.ones(corr_matr.shape), k=1).astype(np.bool))
        collin_cols = [col for col in upper_matr.columns if (upper_matr[col] > threshold).any()]
        return collin_cols
    
    def fit(self, df):
        self._collin_cols = self._collinear_columns(df, self._threshold)
        return self
    
    def transform(self, df):
        all_cols = df.columns.to_list()
        nonexist_cols = [col for col in self._collin_cols if col not in all_cols]
        if len(nonexist_cols) > 0:
            print("WARNING: These collinear cols to be droped do not exist in df:", nonexist_cols)
            
        droped_col = [col for col in self._collin_cols if col in all_cols]
        return df.drop(droped_col, axis="columns")


class OneHotEncoder(BaseEstimator, TransformerMixin):
    def fit(self, train_df):
        df_cat = train_df.select_dtypes(["object", "category"])
        self._cat_cols = df_cat.columns.to_list()
        self._cat_cols_ohe = pd.get_dummies(df_cat).columns.to_list()
    
    def transform(self, df):
        df_cat = df.select_dtypes(["object", "category"])
        cat_cols = df_cat.columns.to_list()
        assert set(cat_cols) == set(self._cat_cols), "df does not have the same categorical cols as train_df"
        
        # one-hot encode
        df_cat = pd.get_dummies(df_cat)
        # drop redundant classes which my be present in test_df
        for col in df_cat.columns:
            if col not in self._cat_cols_ohe:
                df_cat = df_cat.drop([col], axis="columns")
        
        # if some some colums are lacking in test but present in train, make them will all zero 
        cat_cols_ohe = df_cat.columns.to_list()
        for col in self._cat_cols_ohe:
            if col not in cat_cols_ohe:
                df_cat[col] = 0
                df_cat[col] = df_cat[col].astype(np.uint8)
        
        num_cols = [col for col in df.columns if col not in cat_cols]
        df_num = df[num_cols]
        
        return pd.concat([df_num, df_cat], axis="columns")

In [5]:
def flatten_multiindex_cols(columns):
    fat_cols = ["_".join([str(c) for c in flat_col]) for flat_col in columns.to_flat_index()]
    return fat_cols


def agg_num_cols(df, by_sers, stats):
    assert type(by_sers) in [list, tuple], "by_sers must be a list or tuple"
    assert type(stats) in [list, tuple], "stats must be a list or tuple"
    
    for ser in by_sers:
        assert isinstance(ser, pd.Series), "ser in by_sers must be Series"
        
    cat_cols = df.select_dtypes(["object", "bool", "category"]).columns.to_list()
    if len(cat_cols) > 0:
        raise ValueError("There are non-number cols: " + ", ".join(cat_cols))
    
    df_agg = df.groupby(by_sers).agg(stats)
    df_agg.columns = flatten_multiindex_cols(df_agg.columns)
    df_agg = df_agg.reset_index()
    
    return df_agg


def agg_cat_cols(df, by_sers, stats):
    assert type(by_sers) in [list, tuple], "by_sers must be a list or tuple"
    assert type(stats) in [list, tuple], "stats must be a list or tuple"
    
    for ser in by_sers:
        assert isinstance(ser, pd.Series), "ser in by_sers must be Series"
        
    num_cols = df.select_dtypes(["number"]).columns.to_list()
    if len(num_cols) > 0:
        raise ValueError("There are number cols: " + ", ".join(num_cols))
    
    df_agg = df.groupby(by_sers).agg(stats)
    df_agg.columns = flatten_multiindex_cols(df_agg.columns)
    df_agg = df_agg.reset_index()
    
    return df_agg

In [6]:
def mode(ser):
    return ser.mode().values[0]

# Load data

In [8]:
application_train = load_csv("data/download/application_train.csv")
application_test = load_csv("data/download/application_test.csv")

print(application_train.shape, application_test.shape)

appl_train_key = application_train["SK_ID_CURR"]
appl_test_key = application_test["SK_ID_CURR"]

Memory usage before changing types 300.13 MB
Memory usage after changing types 104.87 MB
Memory usage before changing types 47.18 MB
Memory usage after changing types 18.19 MB
(307511, 122) (48744, 121)


In [9]:
bureau = load_csv("data/download/bureau.csv")
print(bureau.shape)

bureau_key = bureau["SK_ID_BUREAU"]

Memory usage before changing types 233.43 MB
Memory usage after changing types 101.27 MB
(1716428, 17)


In [10]:
bureau_balance = load_csv("data/download/bureau_balance.csv")
bureau_balance.shape

Memory usage before changing types 655.20 MB
Memory usage after changing types 245.70 MB


(27299925, 3)

In [11]:
previous_application = load_csv("data/download/previous_application.csv")
print(previous_application.shape)

previous_application_key = previous_application["SK_ID_PREV"]

Memory usage before changing types 494.38 MB
Memory usage after changing types 162.02 MB
(1670214, 37)


In [12]:
POS_CASH_balance = load_csv("data/download/POS_CASH_balance.csv")
POS_CASH_balance.shape

Memory usage before changing types 640.09 MB
Memory usage after changing types 290.04 MB


(10001358, 8)

In [13]:
installments_payments = load_csv("data/download/installments_payments.csv")
installments_payments.shape

Memory usage before changing types 870.75 MB
Memory usage after changing types 435.37 MB


(13605401, 8)

In [14]:
credit_card_balance = load_csv("data/download/credit_card_balance.csv")
credit_card_balance.shape

Memory usage before changing types 706.62 MB
Memory usage after changing types 341.79 MB


(3840312, 23)

# Feature extraction from `application_[train|test]`

In [245]:
def get_colnames_from_regex(df, regex_strings):
    cols = []
    for regex_str in regex_strings:
        cols.extend(df.filter(regex=regex_str).columns.to_list())
    return cols


class ApplImputer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self._regex_strings = ["^APARTMENTS_", "^BASEMENTAREA_", "^YEARS_B", "^COMMONAREA_", 
                               "^ELEVATORS_", "^ENTRANCES_", "^FLOORS", "^LANDAREA_", "^LIVING", 
                               "^NONLIVING", "AMT_REQ_CREDIT_BUREAU_"]
        
        self._spec_impt_vals_num = {"OWN_CAR_AGE": -1.,
                                    "EXT_SOURCE_1": 0.,
                                    "EXT_SOURCE_3": 0.,
                                    "TOTALAREA_MODE": -1.}
    
    def fit(self, df_train):
        cols_imput_with_m1 = get_colnames_from_regex(df_train, self._regex_strings)
        self._spec_impt_vals_num.update({col: -1. for col in cols_imput_with_m1})
        
        df_num = df_train.select_dtypes(["number"])
        self._imputer_num = NumColsImputer(specified_values=self._spec_impt_vals_num, default="median")
        self._imputer_num.fit(df_num)
        
        df_cat = df_train.select_dtypes(["object", "category", "bool"])
        self._imputer_cat = CatColsImputer(specified_values=None, default="missing_value")
        self._imputer_cat.fit(df_cat)
    
    def transform(self, df):
        num_df = df.select_dtypes(["number"])
        
        isnull_df = num_df[["SK_ID_CURR"]]
        for col in self._spec_impt_vals_num:
            isnull_df[col + "_ISNULL"] = num_df[col].isnull()
            
        isnull_df = isnull_df.drop(["SK_ID_CURR"], axis="columns")
        num_df = self._imputer_num.transform(num_df)
        
        # cat
        cat_df = df.select_dtypes(["object", "category", "bool"])
        cat_df = self._imputer_cat.transform(cat_df)
        
        return pd.concat([num_df, isnull_df, cat_df], axis="columns")
        

class ApplNewColsAdder(BaseEstimator, TransformerMixin):
    def fit(self, df_train):
        credit_to_income = df_train["AMT_CREDIT"] / df_train["AMT_INCOME_TOTAL"]
        self._cti_min = credit_to_income.replace(-np.inf, np.nan).min() / 10.
        self._cti_max = credit_to_income.replace(np.inf, np.nan).max() * 10.
        
        credit_to_goods = df_train["AMT_CREDIT"] / df_train["AMT_GOODS_PRICE"]
        self._ctg_min = credit_to_goods.replace(-np.inf, np.nan).min() / 10.
        self._ctg_max = credit_to_goods.replace(np.inf, np.nan).max() / 10.
    
    def transform(self, df):
        df_new = df.copy()
        df_new["AMT_INCOME_TOTAL_LOG"] = np.log(df_new["AMT_INCOME_TOTAL"])
        df_new["DAYS_EMPLOYED_POSITIVE"] = df_new["DAYS_EMPLOYED"] > 0
        days_emp_max = df_new["DAYS_EMPLOYED"].max()
        if days_emp_max > 0:
            df_new["DAYS_EMPLOYED"] = df_new["DAYS_EMPLOYED"].replace({days_emp_max: 100})
        
        df_new["CREDIT_TO_INCOME"] = df_new["AMT_CREDIT"] / df_new["AMT_INCOME_TOTAL"]
        df_new["CREDIT_TO_INCOME"] = df_new["CREDIT_TO_INCOME"].replace(-np.inf, self._cti_min)
        df_new["CREDIT_TO_INCOME"] = df_new["CREDIT_TO_INCOME"].replace(np.inf, self._cti_max)
        
        df_new["CREDIT_TO_GOODS"] = df_new["AMT_CREDIT"] / df_new["AMT_GOODS_PRICE"]
        df_new["CREDIT_TO_GOODS"] = df_new["CREDIT_TO_GOODS"].replace(-np.inf, self._ctg_min)
        df_new["CREDIT_TO_GOODS"] = df_new["CREDIT_TO_GOODS"].replace(np.inf, self._ctg_max)
        
        return df_new
        

def add_cols_application(appl_df):
    appl_df["DAYS_EMPLOYED_POSITIVE"] = appl_df["DAYS_EMPLOYED"] > 0
    days_emp_max = appl_df["DAYS_EMPLOYED"].max()
    appl_df["DAYS_EMPLOYED"] = appl_df["DAYS_EMPLOYED"].replace({days_emp_max: 100})
    appl_df["AMT_INCOME_TOTAL_LOG"] = np.log(appl_df["AMT_INCOME_TOTAL"])
    
    appl_df["CREDIT_TO_INCOME"] = appl_df["AMT_CREDIT"] / appl_df["AMT_INCOME_TOTAL"]
    noinf_min = appl_df["CREDIT_TO_INCOME"].replace(-np.inf, np.nan).min() / 10.
    noinf_max = appl_df["CREDIT_TO_INCOME"].replace(np.inf, np.nan).max() * 10.
    appl_df["CREDIT_TO_INCOME"].replace(-np.inf, noinf_min)
    appl_df["CREDIT_TO_INCOME"].replace(np.inf, noinf_max)
    
    
    appl_df["CREDIT_TO_GOODS"] = appl_df["AMT_CREDIT"] / appl_df["AMT_GOODS_PRICE"]
    noinf_min = appl_df["CREDIT_TO_GOODS"].replace(-np.inf, np.nan).min() / 10.
    noinf_max = appl_df["CREDIT_TO_GOODS"].replace(np.inf, np.nan).max() * 10.
    appl_df["CREDIT_TO_GOODS"].replace(-np.inf, noinf_min)
    appl_df["CREDIT_TO_GOODS"].replace(np.inf, noinf_max)
    return appl_df

In [15]:


"""
df = application_train.copy()

df["DAYS_EMPLOYED_POSITIVE"] = df["DAYS_EMPLOYED"] > 0
df["DAYS_EMPLOYED"] = df["DAYS_EMPLOYED"].replace({days_emp_max: np.nan})
df["AMT_INCOME_TOTAL_LOG"] = np.log(df["AMT_INCOME_TOTAL"])

df["CREDIT_TO_INCOME"] = df["AMT_CREDIT"] / df["AMT_INCOME_TOTAL"]
df["CREDIT_TO_GOODS"] = df["AMT_CREDIT"] / df["AMT_GOODS_PRICE"]
"""


'\ndf = application_train.copy()\n\ndf["DAYS_EMPLOYED_POSITIVE"] = df["DAYS_EMPLOYED"] > 0\ndf["DAYS_EMPLOYED"] = df["DAYS_EMPLOYED"].replace({days_emp_max: np.nan})\ndf["AMT_INCOME_TOTAL_LOG"] = np.log(df["AMT_INCOME_TOTAL"])\n\ndf["CREDIT_TO_INCOME"] = df["AMT_CREDIT"] / df["AMT_INCOME_TOTAL"]\ndf["CREDIT_TO_GOODS"] = df["AMT_CREDIT"] / df["AMT_GOODS_PRICE"]\n'

In [193]:
print(application_train.shape)
application_train["SK_ID_CURR"].head(2)

(307511, 122)


0    100002
1    100003
Name: SK_ID_CURR, dtype: int32

In [194]:
print(application_test.shape)
application_test["SK_ID_CURR"].head(2)

(48744, 121)


0    100001
1    100005
Name: SK_ID_CURR, dtype: int32

In [246]:
appl_imputer = ApplImputer()
appl_imputer.fit(application_train)

df_imput_train = appl_imputer.transform(application_train)
df_imput_test = appl_imputer.transform(application_test)

In [247]:
df_imput_train.shape, df_imput_test.shape

((307511, 174), (48744, 173))

In [125]:
remover = CollinearColumnRemover(0.99, col_regex="_ISNULL$")
remover.fit(df_imput)
#df_imput_remove_collin = remover.transform(df_imput)

CollinearColumnRemover(threshold=None)

In [198]:
df_train = df_imput_train[["NAME_CONTRACT_TYPE", "CODE_GENDER", "NAME_TYPE_SUITE"]]
df_test = df_imput_test[["NAME_CONTRACT_TYPE", "CODE_GENDER", "NAME_TYPE_SUITE"]]

In [206]:
ohe = OneHotEncoder()
ohe.fit(df_imput_train)

df_train_ohe = ohe.transform(df_imput_train)
df_test_ohe = ohe.transform(df_imput_test)

In [209]:
print(df_imput_train.shape)
print(df_train_ohe.shape)
df_train_ohe[["SK_ID_CURR"]].head(2)

(307511, 174)
(307511, 304)


,SK_ID_CURR
0,100002
1,100003


In [210]:
print(df_imput_test.shape)
print(df_test_ohe.shape)
df_test_ohe[["SK_ID_CURR"]].head(2)

(48744, 173)
(48744, 303)


,SK_ID_CURR
0,100001
1,100005


In [237]:
data = pd.DataFrame(np.random.randn(10, 3))
data

,0,1,2
0,-1.037001,0.032839,-1.649569
1,0.148887,-0.304556,0.527149
2,0.365257,0.239204,0.295006
3,-0.424296,-1.536629,0.272708
4,-1.484289,0.653949,2.623585
5,-0.481497,0.987740,0.765191
6,1.214164,-0.320809,-1.048682
7,-0.245094,-0.071874,-0.021049
8,-0.584548,-0.068395,-1.105471
9,0.001568,-0.440571,0.342614


In [238]:
data.loc[3, 0] = np.inf
data.loc[5, 1] = -np.inf
data.loc[8, 2] = np.inf
data

,0,1,2
0,-1.037001,0.032839,-1.649569
1,0.148887,-0.304556,0.527149
2,0.365257,0.239204,0.295006
3,inf,-1.536629,0.272708
4,-1.484289,0.653949,2.623585
5,-0.481497,-inf,0.765191
6,1.214164,-0.320809,-1.048682
7,-0.245094,-0.071874,-0.021049
8,-0.584548,-0.068395,inf
9,0.001568,-0.440571,0.342614


In [241]:
data.replace(np.inf, np.nan).max()

0    1.214164
1    0.653949
2    2.623585
dtype: float64

In [224]:
data.min()

0   -1.232949
1        -inf
2   -2.402992
dtype: float64